In [ ]:
#Change the runtime type into GPU mode before running the code.

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:


import sys, os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras.utils import np_utils
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

##Change the file location properly.copy the path by right clicking on the csv file in drive.

df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/fer2013.csv')

print(df.info())
print(df["Usage"].value_counts())

print(df.head(10))




Using TensorFlow backend.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None
Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
5        2  55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...  Training
6        4  20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 

In [ ]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)


#normalizing data between 0 and 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

# print(f"shape:{X_train.shape}")

## Designing the CNN
# 1st convolution layer

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))

# model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

# 2nd convolution layer

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

# model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))

# model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Flatten())

# fully connected neural networks

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])
print("Compilation Done!!")





Compilation Done!!


In [ ]:
#Training the model

model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)




Epoch 1/100
449/449 [==============================] - 18s 40ms/step - loss: 1.7020 - accuracy: 0.3095 - val_loss: 1.5092 - val_accuracy: 0.4007
Epoch 2/100
449/449 [==============================] - 18s 40ms/step - loss: 1.4784 - accuracy: 0.4214 - val_loss: 1.3822 - val_accuracy: 0.4726
Epoch 3/100
449/449 [==============================] - 18s 40ms/step - loss: 1.3762 - accuracy: 0.4648 - val_loss: 1.3038 - val_accuracy: 0.4962
Epoch 4/100
449/449 [==============================] - 18s 40ms/step - loss: 1.3194 - accuracy: 0.4918 - val_loss: 1.2674 - val_accuracy: 0.5116
Epoch 5/100
449/449 [==============================] - 18s 40ms/step - loss: 1.2819 - accuracy: 0.5011 - val_loss: 1.2333 - val_accuracy: 0.5302
Epoch 6/100
449/449 [==============================] - 18s 40ms/step - loss: 1.2388 - accuracy: 0.5278 - val_loss: 1.2175 - val_accuracy: 0.5325
Epoch 7/100
449/449 [==============================] - 18s 40ms/step - loss: 1.2156 - accuracy: 0.5357 - val_loss: 1.1964 - val_ac

In [ ]:
#Saving the  model 
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("/content/gdrive/My Drive/Colab Notebooks/ferj.h5")

In [ ]:
## Loading the model

from keras.models import load_model

classifier = model.load_weights('/content/gdrive/My Drive/Colab Notebooks/ferj.h5')